Aditiya Artadinata / 2201735640

# Library

In [1]:
!pip install Sastrawi
!pip install transformers

from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model_name = 'bert-base-uncased'
model_name = 'cahya/bert-base-indonesian-522M'

     |████████████████████████████████| 215kB 21.7MB/s 
     |████████████████████████████████| 1.8MB 16.7MB/s 
     |████████████████████████████████| 3.2MB 48.2MB/s 
     |████████████████████████████████| 890kB 45.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=aa88f52170b07f4534013379b67950e9d8370947ccaf66f870e1530af3ed2f0b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!wget 'https://drive.google.com/uc?export=download&id=1GALJaSIBZQk6DHjwcXPzyfBgvXBTEmWf' -q -O hatespeech.txt
!wget 'https://drive.google.com/uc?export=download&id=1uK_1xuk4E_nJWPSa-H58GnaR-8n43G9e' -q -O sentiment.csv

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import string
import re

%matplotlib inline
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
def convert_data_to_examples(train, valid, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, 
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = valid.apply(lambda x: InputExample(guid=None, 
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  test_InputExamples = test.apply(lambda x: InputExample(guid=None, 
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples, test_InputExamples

def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []

    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, 
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, 
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

# Sentiment.csv

In [6]:
df_sentiment = pd.read_csv('sentiment.csv', sep=';')

df_sentiment

,Tweets,Label
0,rt @mrtampi: agus makin santai.\nahok makin sa...,negatif
1,pilkada dki jangan pilih pki!! berbahaya!! pil...,negatif
2,pdip sengaja becah belah rakyat warga dki tida...,negatif
3,rt @gunromli: sylviana kesyikan ngomong sendir...,negatif
4,rt @gunromli: sylviana kesyikan ngomong sendir...,negatif
...,...,...
1501,rt @zul_hasan: sebelum debat kompak dukung agu...,positif
1502,"#debat2pilkadadki apik kabeh, yo seng adil men...",positif
1503,rt @erixputra: rakyat adalah bos kami. kami ad...,positif
1504,ahok \u2013 djarot waspadai politik uang dalam...,positif


In [7]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize

sw_factory = StopWordRemoverFactory()
stopword = sw_factory.create_stop_word_remover()

stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

kalimat = "debat kok saling menjatuhkan ki pie dua dua nya nyerang eh malah bertengkar sendiri yg di serang melerai ngakak  d"
kalimat = kalimat.translate(str.maketrans('','',string.punctuation)).lower()
stop = stopword.remove(kalimat)
print(kalimat)
print(stop)

debat kok saling menjatuhkan ki pie dua dua nya nyerang eh malah bertengkar sendiri yg di serang melerai ngakak  d
debat kok saling menjatuhkan ki pie dua nya nyerang eh malah bertengkar sendiri yg serang melerai ngakak  d


Pre-process untuk menghapus links, mentions(@), hashtag(#), single letters, hasil emote(\u, \n), digit, dan tanda baca

In [8]:
def clean_text(text):
  text = text.lower()
  text = re.compile('#\w+ ').sub('', re.compile('rt @\w+: ').sub('', text, count=1)).strip()
  text = re.sub(r"http\S+", "", text)
  text = re.sub(r'@\w+', '', text)
  text = re.sub(r'#debat\w+', '', text)
  text = text.strip().replace('\\n', '')
  text = re.sub(r'\\u\w+', '', text)
  text = re.sub('\d+', '', text)
  text = re.sub(r"[,.;@#?!&$]+\ *", " ", text)
  text = "".join([char for char in text if char not in string.punctuation])
  text = re.sub(r'\b\w\b', '', text)
  text = stopword.remove(text)
  text = stemmer.stem(text)
  
  return text

def text_cleaner(text):
  text = text.lower()
  text = re.compile('#\w+ ').sub('', re.compile('rt @\w+: ').sub('', text, count=1)).strip()
  text = re.sub(r"http\S+", "", text)
  text = re.sub(r'@\w+', '', text)
  text = re.sub(r'#debat\w+', '', text)
  text = text.strip().replace('\\n', '')
  text = re.sub(r'\\u\w+', '', text)
  text = re.sub('\d+', '', text)
  text = re.sub(r"[,.;@#?!&$]+\ *", " ", text)
  text = "".join([char for char in text if char not in string.punctuation])
  text = stopword.remove(text)
  text = stemmer.stem(text)
  
  return text

## Pre-process

In [9]:
df_sentiment['clean'] = df_sentiment['Tweets'].apply(lambda x: clean_text(x))

df_sentiment.loc[df_sentiment.Label=='negatif', 'Label'] = 0
df_sentiment.loc[df_sentiment.Label=='positif', 'Label'] = 1

df_sentiment.to_csv('prep_sentiment.csv', index=False)
df_sentiment

,Tweets,Label,clean
0,rt @mrtampi: agus makin santai.\nahok makin sa...,0,agus makin santai ahok makin santun anies maki...
1,pilkada dki jangan pilih pki!! berbahaya!! pil...,0,pilkada dki jangan pilih pki berbahaya pilih ...
2,pdip sengaja becah belah rakyat warga dki tida...,0,pdip sengaja becah belah rakyat warga dki tida...
3,rt @gunromli: sylviana kesyikan ngomong sendir...,0,sylviana kesyikan ngomong sendiri lupa bertany...
4,rt @gunromli: sylviana kesyikan ngomong sendir...,0,sylviana kesyikan ngomong sendiri lupa bertany...
...,...,...,...
1501,rt @zul_hasan: sebelum debat kompak dukung agu...,1,sebelum debat kompak dukung agus sylvi setela...
1502,"#debat2pilkadadki apik kabeh, yo seng adil men...",1,apik kabeh yo seng adil mending ngko pas pemil...
1503,rt @erixputra: rakyat adalah bos kami. kami ad...,1,rakyat adalah bos kami kami adalah pelayan rak...
1504,ahok \u2013 djarot waspadai politik uang dalam...,1,ahok djarot waspadai politik uang dalam pilkada


In [10]:
df_sentiment['clean'][25]

'gitu yg mau jadi pemimpin udah salah harusnya intropeksi diri bukan ngacuin jempol kebawah gitu emak sylvii '

In [11]:
df_sentiment['Label'].value_counts()

1    757
0    749
Name: Label, dtype: int64

In [12]:
train_sen, test_sen = np.split(df_sentiment.sample(frac=1, random_state=10), [int(0.6*len(df_sentiment))])
test_sen, valid_sen = np.split(test_sen.sample(frac=1, random_state=10), [int(0.05*len(test_sen))])

In [13]:
print(f'train_sen : {train_sen.shape}')
print(f'test_sen  : {test_sen.shape}')
print(f'valid_sen : {valid_sen.shape}')

train_sen : (903, 3)
test_sen  : (30, 3)
valid_sen : (573, 3)


In [14]:
train_sen_InputExamples, valid_sen_InputExamples, test_sen_InputExamples = convert_data_to_examples(train_sen, valid_sen, test_sen, 'clean', 'Label')

## Train model

In [15]:
tensorboard_callback_sentiment = tf.keras.callbacks.TensorBoard(log_dir="logs_sen")
model_sentiment = TFBertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

train_sen_data = convert_examples_to_tf_dataset(list(train_sen_InputExamples), tokenizer)
train_sen_data = train_sen_data.shuffle(100).batch(32).repeat(2)

valid_sen_data = convert_examples_to_tf_dataset(list(valid_sen_InputExamples), tokenizer)
valid_sen_data = valid_sen_data.batch(32)

test_sen_data = convert_examples_to_tf_dataset(list(test_sen_InputExamples), tokenizer)
test_sen_data = test_sen_data.batch(32)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-522M and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model_sentiment.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model_sentiment.fit(train_sen_data, epochs=2, validation_data=valid_sen_data, callbacks=[tensorboard_callback_sentiment])

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fe6dd4e62a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fe6dd4e62a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert

Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the functio

### Evaluation

In [36]:
from sklearn.metrics import classification_report

def model_eval(model, test_data, test_label, verbose=False):
  tf_batch = tokenizer(test_data, max_length=128, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)

  labels = ['NEGATIVE', 'POSITIVE']
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()

  # Print all Text and Prediction
  if verbose:
    for i in range(len(test_data)):
      print('[', i+1, ']', test_data[i], ": ", labels[label[i]])

  print(f'\nPrediction:\n{label.tolist()}')
  print(f'True label:\n{test_label}')

  if verbose: return

  rep = classification_report(test_label, label)
  print(f'\nClassification Report:\n{rep}')

In [18]:
test_sen_list = test_sen['clean'].head(test_sen.shape[0]).values.tolist()
test_sen_label = test_sen['Label'].head(test_sen.shape[0]).values.tolist()
test_sen.head()

,Tweets,Label,clean
532,vivacoid: ini video saat nusron wahid kritik m...,0,vivacoid ini video saat nusron wahid kritik mu...
254,rt @cagubnyinyir: setara institut nilai mui be...,0,setara institut nilai mui berpolitik tdk sehat
453,rt @batack_overkill: #debat2pilkadadki paslon ...,0,paslon akan menaikkan gaji guru paud jika te...
1497,#debat2pilkadadki 1 dan 3 sebaiknya pahami dul...,1,dan sebaiknya pahami dulu undangundang dan h...
848,rt @ochibi19: jelas pk ahok paling brkemampuan...,1,jelas pk ahok paling brkemampuan jd gubernur k...


In [19]:
model_eval(model_sentiment, test_sen_list, test_sen_label)


Prediction:
[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0]
True label:
[0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0]

Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.60      0.62        15
           1       0.62      0.67      0.65        15

    accuracy                           0.63        30
   macro avg       0.63      0.63      0.63        30
weighted avg       0.63      0.63      0.63        30



# hatespeech.txt

In [20]:
df_hatespeech = pd.read_csv('hatespeech.txt', sep='\t', engine='python')

df_hatespeech

,Label,Tweet
0,Non_HS,RT @spardaxyz: Fadli Zon Minta Mendagri Segera...
1,Non_HS,RT @baguscondromowo: Mereka terus melukai aksi...
2,Non_HS,Sylvi: bagaimana gurbernur melakukan kekerasan...
3,Non_HS,"Ahmad Dhani Tak Puas Debat Pilkada, Masalah Ja..."
4,Non_HS,RT @lisdaulay28: Waspada KTP palsu.....kawal P...
...,...,...
708,HS,Muka Si BABi Ahok Tuh Yg Mirip SERBET Lantai.....
709,HS,"Betul bang hancurkan merka bang, musnahkan chi..."
710,HS,"Sapa Yg bilang Ahok anti korupsi!?, klo grombo..."
711,HS,"Gw juga ngimpi SENTILIN BIJI BABI AHOK, pcetar..."


## Pre-process

In [21]:
df_hatespeech['clean'] = df_hatespeech['Tweet'].apply(lambda x: clean_text(x))

df_hatespeech.loc[df_hatespeech.Label=='HS', 'Label'] = 0
df_hatespeech.loc[df_hatespeech.Label=='Non_HS', 'Label'] = 1

df_hatespeech.to_csv('prep_hatespeech.csv', index=False)
df_hatespeech

,Label,Tweet,clean
0,1,RT @spardaxyz: Fadli Zon Minta Mendagri Segera...,fadli zon minta mendagri segera menonaktifkan ...
1,1,RT @baguscondromowo: Mereka terus melukai aksi...,mereka terus melukai aksi dalam rangka memenja...
2,1,Sylvi: bagaimana gurbernur melakukan kekerasan...,sylvi bagaimana gurbernur melakukan kekerasan ...
3,1,"Ahmad Dhani Tak Puas Debat Pilkada, Masalah Ja...",ahmad dhani tak puas debat pilkada masalah jal...
4,1,RT @lisdaulay28: Waspada KTP palsu.....kawal P...,waspada ktp palsu kawal pilkada
...,...,...,...
708,0,Muka Si BABi Ahok Tuh Yg Mirip SERBET Lantai.....,muka si babi ahok tuh yg mirip serbet lantai
709,0,"Betul bang hancurkan merka bang, musnahkan chi...",betul bang hancurkan merka bang musnahkan chin...
710,0,"Sapa Yg bilang Ahok anti korupsi!?, klo grombo...",sapa yg bilang ahok anti korupsi klo grombolan...
711,0,"Gw juga ngimpi SENTILIN BIJI BABI AHOK, pcetar...",gw juga ngimpi sentilin biji babi ahok pcetar ...


In [22]:
df_hatespeech['Label'].value_counts()

1    453
0    260
Name: Label, dtype: int64

In [23]:
train_hs, test_hs = np.split(df_hatespeech.sample(frac=1, random_state=1), [int(0.6*len(df_hatespeech))])
test_hs, valid_hs = np.split(test_hs.sample(frac=1, random_state=5), [int(0.1*len(test_hs))])

In [24]:
print(f'train_sen : {train_hs.shape}')
print(f'test_sen  : {test_hs.shape}')
print(f'valid_sen : {valid_hs.shape}')

train_sen : (427, 3)
test_sen  : (28, 3)
valid_sen : (258, 3)


In [25]:
train_hs_InputExamples, valid_hs_InputExamples, test_hs_InputExamples = convert_data_to_examples(train_hs, valid_hs, test_hs, 'clean', 'Label')

## Train Model

In [26]:
tensorboard_callback_hs = tf.keras.callbacks.TensorBoard(log_dir="logs_hs")
model_hatespeech = TFBertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

train_hs_data = convert_examples_to_tf_dataset(list(train_hs_InputExamples), tokenizer)
train_hs_data = train_hs_data.shuffle(100).batch(32).repeat(2)

valid_hs_data = convert_examples_to_tf_dataset(list(valid_hs_InputExamples), tokenizer)
valid_hs_data = valid_hs_data.batch(32)

test_hs_data = convert_examples_to_tf_dataset(list(test_hs_InputExamples), tokenizer)
test_hs_data = test_hs_data.batch(32)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-522M and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [27]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model_hatespeech.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model_hatespeech.fit(train_hs_data, epochs=2, validation_data=valid_hs_data, callbacks=[tensorboard_callback_hs])

Epoch 1/2
     28/Unknown - 33s 857ms/step - loss: 0.5513 - accuracy: 0.6942WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
28/28 [==============================] - 37s 994ms/step - loss: 0.5464 - accuracy: 0.6979 - val_loss: 0.3783 - val_accuracy: 0.8643
Epoch 2/2
28/28 [==============================] - 26s 926ms/step - loss: 0.1742 - accuracy: 0.9325 - val_loss: 0.4141 - val_accuracy: 0.8798


### Evaluation

In [28]:
test_hs_list = test_hs['clean'].head(test_hs.shape[0]).values.tolist()
test_hs_label = test_hs['Label'].head(test_hs.shape[0]).values.tolist()
test_hs.head()

,Label,Tweet,clean
465,0,#DebatFinalPilkadaJKT mampuuuusssssss mpoK Sil...,mampuuuusssssss mpok silvy
561,0,Perilaku Si BABI @jokoanwar ini memang benci d...,perilaku si babi ini memang benci dgn hal ber...
461,0,@YohanesTarigan9 saya ga habis pikir kenapa fa...,saya ga habis pikir kenapa fanboy ahok bego be...
711,0,"Gw juga ngimpi SENTILIN BIJI BABI AHOK, pcetar...",gw juga ngimpi sentilin biji babi ahok pcetar ...
599,0,RT @Ra_Ria_Rana: GAK HABIS PIKIR???\n\nTINGGAL...,gak habis pikir tinggal nunggu adzab allah nih...


In [29]:
model_eval(model_hatespeech, test_hs_list, test_hs_label)


Prediction:
[0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0]
True label:
[0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.64      0.78        14
           1       0.74      1.00      0.85        14

    accuracy                           0.82        28
   macro avg       0.87      0.82      0.82        28
weighted avg       0.87      0.82      0.82        28



# Cross-Inferencing

## Sentiment model

### Sentiment data

In [39]:
model_eval(model_sentiment, test_sen_list, test_sen_label, True)

[ 1 ] vivacoid ini video saat nusron wahid kritik mui soal ahok :  NEGATIVE
[ 2 ] setara institut nilai mui berpolitik tdk sehat :  NEGATIVE
[ 3 ] paslon   akan menaikkan gaji guru paud jika terpilih waktu jadi mendikbud ente ngapain bray tidu :  NEGATIVE
[ 4 ]  dan  sebaiknya pahami dulu undangundang dan hukum pemerintahan daerah setelah itu pelajari data statistik  :  POSITIVE
[ 5 ] jelas pk ahok paling brkemampuan jd gubernur klo yg kalian prmslahkan itu pribadinya kalian cuma gak kenal sja asli baik de :  POSITIVE
[ 6 ] agus jawabnya muter ah ga tajem tepat sasaran  pic  :  POSITIVE
[ 7 ] rachmawati tersinggung megawati ajak ahok ke makam bung karno   riau :  POSITIVE
[ 8 ] keliatan kok najwa yg gelisah smua pertanyaan dijawab sg cerdas oleh mas agus jawab cerdas dan akurat kok dibilang emosi :  POSITIVE
[ 9 ] lihat pa ditangkap atas laporan warga tapi btp bebas padahal ada laporan bpk   :  NEGATIVE
[ 10 ] membangun tanpa menyakitianiessandi cerdas  santun  amp :  NEGATIVE
[ 11 ] k

### Hatespeech data


Dengan menggunakan model Sentiment pada data Hatespeech dapat terlihat bahwa model banyak menentukan tweets dengan label HS mempunyai sentiment yang negatif. Tidak hanya itu ada juga tweets dengan label NON_HS mempunyai kesan yang negatif seperti pada tweets [ 6 ] dan [ 25 ].

In [40]:
model_eval(model_sentiment, test_hs_list, test_hs_label, True)

[ 1 ] mampuuuusssssss mpok silvy  :  NEGATIVE
[ 2 ] perilaku si babi  ini memang benci dgn hal berbau islam cocok dgn ahok  :  NEGATIVE
[ 3 ] saya ga habis pikir kenapa fanboy ahok bego bego ya pada ga ngerti sejarah gampang kejebak iklan maaf ya  :  NEGATIVE
[ 4 ] gw juga ngimpi sentilin biji babi ahok pcetar pcetar langsung kejang die ngadu ke jkw jkw makanye sunat dulu bru ngomong almaidah  :  NEGATIVE
[ 5 ] gak habis pikir tinggal nunggu adzab allah nih negeri pemimpinnya sdh dzalim :  POSITIVE
[ 6 ] itu yang ngomong jangan pilih ahok djarot pernah ngerasain banjir ga sih  :  NEGATIVE
[ 7 ] mana bisa percya lagi sdgkan induknya penista smp skrg lum ditahan tp malah dibela  :  NEGATIVE
[ 8 ] dan selamat atas kemenangannya untuk bapak anies  sandi semoga jakarta bisa bersatu lagi :  POSITIVE
[ 9 ] mudahhan pak ahok jadi nomor satu di indonesia :  POSITIVE
[ 10 ] jangan berpihak melayani rakyat dengan sepenuh hati tuhan yang memberi kekuasaan tuhan yang mengambil ahok matanajwadebatja

## Hatespeech model

### Sentiment data

Dengan model Hatespeech pada data Sentment, terdapat beberapa tweets yang mempunyai label Negatif namun model menganggap bahwa tweets tersebut bukan termasuk dalam HS ([ 27 ], [ 6 ], [ 7 ]).

In [41]:
model_eval(model_hatespeech, test_sen_list, test_sen_label, True)

[ 1 ] vivacoid ini video saat nusron wahid kritik mui soal ahok :  POSITIVE
[ 2 ] setara institut nilai mui berpolitik tdk sehat :  POSITIVE
[ 3 ] paslon   akan menaikkan gaji guru paud jika terpilih waktu jadi mendikbud ente ngapain bray tidu :  POSITIVE
[ 4 ]  dan  sebaiknya pahami dulu undangundang dan hukum pemerintahan daerah setelah itu pelajari data statistik  :  POSITIVE
[ 5 ] jelas pk ahok paling brkemampuan jd gubernur klo yg kalian prmslahkan itu pribadinya kalian cuma gak kenal sja asli baik de :  POSITIVE
[ 6 ] agus jawabnya muter ah ga tajem tepat sasaran  pic  :  POSITIVE
[ 7 ] rachmawati tersinggung megawati ajak ahok ke makam bung karno   riau :  POSITIVE
[ 8 ] keliatan kok najwa yg gelisah smua pertanyaan dijawab sg cerdas oleh mas agus jawab cerdas dan akurat kok dibilang emosi :  POSITIVE
[ 9 ] lihat pa ditangkap atas laporan warga tapi btp bebas padahal ada laporan bpk   :  POSITIVE
[ 10 ] membangun tanpa menyakitianiessandi cerdas  santun  amp :  POSITIVE
[ 11 ] k

### Hatespeech data

In [42]:
model_eval(model_hatespeech, test_hs_list, test_hs_label, True)

[ 1 ] mampuuuusssssss mpok silvy  :  NEGATIVE
[ 2 ] perilaku si babi  ini memang benci dgn hal berbau islam cocok dgn ahok  :  NEGATIVE
[ 3 ] saya ga habis pikir kenapa fanboy ahok bego bego ya pada ga ngerti sejarah gampang kejebak iklan maaf ya  :  POSITIVE
[ 4 ] gw juga ngimpi sentilin biji babi ahok pcetar pcetar langsung kejang die ngadu ke jkw jkw makanye sunat dulu bru ngomong almaidah  :  NEGATIVE
[ 5 ] gak habis pikir tinggal nunggu adzab allah nih negeri pemimpinnya sdh dzalim :  POSITIVE
[ 6 ] itu yang ngomong jangan pilih ahok djarot pernah ngerasain banjir ga sih  :  POSITIVE
[ 7 ] mana bisa percya lagi sdgkan induknya penista smp skrg lum ditahan tp malah dibela  :  POSITIVE
[ 8 ] dan selamat atas kemenangannya untuk bapak anies  sandi semoga jakarta bisa bersatu lagi :  POSITIVE
[ 9 ] mudahhan pak ahok jadi nomor satu di indonesia :  POSITIVE
[ 10 ] jangan berpihak melayani rakyat dengan sepenuh hati tuhan yang memberi kekuasaan tuhan yang mengambil ahok matanajwadebatja

## Kesimpulan


Kesimpulan yang dapat diambil adalah model Sentiment mampu untuk menentukan bahwa hampir dari seluruh data Hatespeech dengan label HS mempunyai sentimen yang negatif. Sentimen negatif dapat berasal dari kata-kata yang menghina atau bersifat negatif. Karena kebanyakan dari tweets HS menggunakan kata-kata yang menghina, maka model menganggap tweets tersebut mempunyai sentimen yang negatif.

Pada model Hatespeech dengan data Sentiment dapat terlihat bahwa beberapa tweets dengan label Negatif bukan merupakan hate speech. Tweets dengan label HS lebih cenderung menggunakan kata-kata yang menghina atau berkesan negatif. Seperti pada contoh, sentimen negatif dapat berasal dari suatu tweets yang berbentuk kritik, yang membedakan tweets tersebut dengan tweets HS adalah penggunaan kata-kata yang menghina.